<a href="https://colab.research.google.com/github/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter/blob/main/ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-6e247b23-2bb3-e804-d578-5c965005fd37)


#install pacakges

In [6]:
%%capture
!pip install ktrain


# download data

In [3]:
import os
if os.path.isdir('/content/Profiling-Hate-Speech-Spreaders-on-Twitter') is False:
    !git clone https://github.com/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter.git

Cloning into 'Profiling-Hate-Speech-Spreaders-on-Twitter'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 86 (delta 24), reused 30 (delta 7), pack-reused 0
Unpacking objects: 100% (86/86), done.


In [7]:
with open('pass.txt','r') as f:
  pas=f.read()

In [8]:
import zipfile
if os.path.isdir('train') is False:
  train_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/train.zip'
  with zipfile.ZipFile(train_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))
if os.path.isdir('test') is False:
  test_path='/content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/test.zip'
  with zipfile.ZipFile(test_path,"r") as zip_ref:
    zip_ref.extractall(".",pwd = bytes(pas,'utf-8'))

# prepare data

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
df=pd.read_csv('train/en/truth.txt',sep=':::',header=None,engine='python')
df.columns=['id','label']

In [10]:
df.label.value_counts()

1    100
0    100
Name: label, dtype: int64

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
def preprocessing(text):
    text=text.str.replace('\d+', '')
    text=text.str.replace('RT','')
    text=text.str.replace('#USER#','')
    text=text.str.replace('#URL#','')
    text= text.str.lower()
    text = text.str.replace('[^\w\s]','')
    #text = text.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
    #text = text.apply(lambda x: [item for item in x if item not in stop])
    #text = text.apply(lambda x : " ".join(x))
    return text

In [13]:
df.id='train/en/'+df.id.astype(str)+'.xml'
df.head()

,id,label
0,train/en/639b8e5e6a527d494c85d8f5704b1a01.xml,0
1,train/en/f2b1fc84c500c38a93522efbd422b559.xml,0
2,train/en/10b2d013382e1fb3c9414ea28329f258.xml,0
3,train/en/26644d1348fc1122e8c5ef45d6bc84fa.xml,0
4,train/en/4d4c5dcbfe38d0d33a0d1b1419952ca8.xml,0


In [14]:
len(df)

200

In [15]:
import xml.etree.ElementTree as ET
def reader(df,ground=True):
  data=[]
  for x in df.iterrows():
      
      tree = ET.parse(x[1].id)
      root = tree.getroot()
      text=[x.text for x in root[0]]
      if ground:
        label=[x[1].label]*len(text)
        data.append(pd.DataFrame(zip(text,label),columns=['text','label']))
      else:
        data.append(pd.DataFrame(text,columns=['text']))
  return data

In [16]:
data=reader(df)

In [17]:
data[0].head()

,text,label
0,RT #USER#: Funny how “15 days to slow the spre...,0
1,RT #USER#: Why did Minneapolis just give Georg...,0
2,"RT #USER#: To be fair, he has done a lot of un...",0
3,RT #USER#: President Trump got us the #HASHTAG...,0
4,RT #USER#: Is the case against former officer ...,0


In [18]:
len(data),len(data[0])

(200, 200)

In [19]:
from sklearn.model_selection import train_test_split
trainx,valx=train_test_split(data,test_size=0.1,)
trainx,testx=train_test_split(trainx,test_size=0.1)

In [20]:
train=pd.concat(trainx)
val=pd.concat(valx)
test=pd.concat(testx)

In [21]:
train.label.value_counts()

1    16600
0    15800
Name: label, dtype: int64

In [22]:
val.label.value_counts()

0    2400
1    1600
Name: label, dtype: int64

In [23]:
test.label.value_counts()

1    1800
0    1800
Name: label, dtype: int64

In [24]:
train.text=preprocessing(train.text)
val.text=preprocessing(val.text)
test.text=preprocessing(test.text)

In [25]:
len(train),len(val),len(test)

(32400, 4000, 3600)

In [26]:
train = train.sample(frac=1).reset_index(drop=True)
val = val.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)


In [27]:
train_text=train.text.values
train_label=train.label.values
val_text=val.text.values
val_label=val.label.values
test_text=test.text.values
test_label=test.label.values

# classify data

In [30]:
from sklearn.model_selection import StratifiedKFold
from ktrain import text
import ktrain
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=20,classes=[1,0])
trn = t.preprocess_train(train_text, train_label)
tst = t.preprocess_test(val_text, val_label)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=tst, batch_size=16)
learner.fit_onecycle(0.00001, 5) 
print(learner.validate(class_names=t.get_classes()))

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 20
	99percentile : 24


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 20
	99percentile : 24




begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
2025/2025 [==============================] - 449s 209ms/step - loss: 0.6843 - accuracy: 0.5444 - val_loss: 0.7108 - val_accuracy: 0.5063
Epoch 2/5
2025/2025 [==============================] - 433s 210ms/step - loss: 0.6478 - accuracy: 0.6115 - val_loss: 0.7050 - val_accuracy: 0.5197
Epoch 3/5
2025/2025 [==============================] - 430s 208ms/step - loss: 0.5887 - accuracy: 0.6736 - val_loss: 0.7878 - val_accuracy: 0.4910
Epoch 4/5
2025/2025 [==============================] - 430s 208ms/step - loss: 0.4593 - accuracy: 0.7752 - val_loss: 1.0284 - val_accuracy: 0.4798
Epoch 5/5
2025/2025 [==============================] - 430s 208ms/step - loss: 0.3278 - accuracy: 0.8550 - val_loss: 1.2204 - val_accuracy: 0.4925
              precision    recall  f1-score   support

           1       0.61      0.43      0.50      2400
           0       0.41      0.59      0.48      1600

    accuracy                          

In [41]:
val = t.preprocess_test(val_text, val_label)

val_pred = learner.predict(val)
print(classification_report(val_label,np.argmax(val_pred,axis=1)))

preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 20
	99percentile : 24


              precision    recall  f1-score   support

           0       0.61      0.43      0.50      2400
           1       0.41      0.59      0.48      1600

    accuracy                           0.49      4000
   macro avg       0.51      0.51      0.49      4000
weighted avg       0.53      0.49      0.49      4000



In [42]:
test = t.preprocess_test(test_text, test_label)

val_pred = learner.predict(test)
print(classification_report(test_label,np.argmax(val_pred,axis=1)))

preprocessing test...
language: en
test sequence lengths:
	mean : 11
	95percentile : 20
	99percentile : 24


              precision    recall  f1-score   support

           0       0.54      0.44      0.49      1800
           1       0.53      0.63      0.57      1800

    accuracy                           0.53      3600
   macro avg       0.54      0.53      0.53      3600
weighted avg       0.54      0.53      0.53      3600



# validation

In [ ]:
preds,trues=[],[]
for i in valx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

In [ ]:
preds,trues=[],[]
for i in testx:
  pred=list(clf.predict(i.text.values,verbose=0))
  preds.append(max(pred,key=pred.count))
  trues.append(int(np.mean(i.label)))
print(classification_report(trues,preds))  

# test evaluation

In [ ]:
from glob import glob
test_path=pd.DataFrame(glob('test/en/*.xml'),columns=['id'])
test_path.head()

,id
0,test/en/f2973063f16c0698a6de86c55b5f9ad6.xml
1,test/en/be71e41921653e523aceb555727b61c7.xml
2,test/en/d0235c3e74f79298fc2b6d2427c2060b.xml
3,test/en/314d4bf068314367bc83dc5ae3757e13.xml
4,test/en/fdd27881da241cff7115c1e89caba6dd.xml


In [ ]:
test=reader(test_path,ground=False)

In [ ]:
test[0].head()

,text
0,#USER# I'm so sorry 😞...
1,#USER# #USER# That is correct.
2,RT #USER#: The legitimate President of the Uni...
3,#USER# #USER# #USER# Texas AG is suing the Bid...
4,#USER# In Jesus Holy Name. Soon the entire wor...


In [ ]:
final_preds=[]
for i in range(len(test)):
  test[i].text=preprocessing(test[i].text)
  pred=list(clf.predict(test[i].text.values,verbose=0))
  final_preds.append(int(max(pred,key=pred.count)))


In [ ]:
len(final_preds),len(test_path)

In [ ]:
ids=test_path.id.apply(lambda x:x.split('/')[-1].split('.')[0])

In [ ]:
lang=['en']*len(test_path)

In [ ]:
final=pd.DataFrame(zip(ids,lang,final_preds),columns=['author-id','lang','type'])
final.head()